# Analysis of the Datapoints Dataframe

## Imports and reading


In [27]:
import pandas as pd
from pathlib import Path
from utils import print_pretty_df

# Quick ANSI color code shortcuts
r = "\033[31m"
y = "\033[33m"
g = "\033[32m"
b = "\033[34m"
e = "\033[0m"

pickleName = "all_datapoints.pkl"
datapointsDfPath = Path("..") / ".." / "data" / "Review_ML-RS-FPGA" / "Dataframes" / pickleName
datapointsDf = pd.read_pickle(datapointsDfPath)

In [28]:
# print(len(datapointsDf))
# print(len(datapointsDf[datapointsDf["Task"].isin(["Classification", "Object detection"])]))
print_pretty_df(datapointsDf)

+----+-------------------------------------------------------+-------------------------------+------------------+----------------+----------------+---------------------------------------------------+----------------------+-----------------------------------------+------------------------------------------------+----------------------+------------------+-----------+-------------+-----------+--------------+-------------------+------------+-------------+-------------------------------------+----------------+-----------------------------------------+-------------------------------------------------------------+-------------------------------------------+------------+----------+----------+---------------------+
|    |                   BBT Citation Key                    |             Model             | Equivalent model |    Backbone    |    Modality    |                      Dataset                      |         Task         |               Application               |                    

## Quick statistics / Overview


### Implementation means

Typical implementation tags look like `'RTL design (VHDL)'` or `'Vitis AI (1.4)'`.
When grouping them by "family" I discard information about language or version.


In [29]:
# --- Initial raw count ---
implementationCounts = datapointsDf["Implementation"].value_counts()
print(implementationCounts)
print()

# -- Group by "family", i.e., discard version or language information in between parenthesis ---
def determine_impl_group(index: str) -> str:
    return index.split("(")[0].strip()
implementationGrouped = implementationCounts.groupby(determine_impl_group).sum()
print(implementationGrouped.sort_values(ascending=False))

Implementation
RTL design (VHDL)       12
N/A                      8
RTL design (Verilog)     7
RTL design (N/A)         7
HLS (Vitis)              6
Vitis AI (N/A)           6
FINN                     4
Vitis AI (DNNDK)         4
HLS (N/A)                3
RTL design (XSG)         3
Vitis AI (v2.5)          1
Vitis AI (v1.4)          1
HLS (Vivado)             1
HLS (MATLAB)             1
HLS (VGT)                1
Name: count, dtype: int64

Implementation
RTL design    29
HLS           12
Vitis AI      12
N/A            8
FINN           4
Name: count, dtype: int64


### FPGA boards

A typical board tag looks like that: `'Zynq 7000 (Z7020) {Arty Z7}'`: `'<family> (<model>) {<evaluation board>}'`.

The following cell groups the board by these 3 criteria.


In [30]:
# --- Initial raw count (of the full tags) ---
boardCounts = datapointsDf["Board"].value_counts()
# print(boardCounts)
# print()

# -- Group by "family", i.e., discard model in between parenthesis and evaluation board in between brackets ---
def determine_board_family_group(index: str) -> str:
    return index.split("(")[0].strip()
boardFamilyGrouped = boardCounts.groupby(determine_board_family_group).sum()
print(boardFamilyGrouped.sort_values(ascending=False))
print()

# -- Group by "board specific model", i.e., discard family and evaluation name in between brackets ---
def determine_board_model_group(index: str) -> str:
    return index.split("(")[1].split("{")[0].strip()[:-1]
boardModelGrouped = boardCounts.groupby(determine_board_model_group).sum()
print(boardModelGrouped.sort_values(ascending=False))
print()

# -- Group by "evaluation board/kit" (the name in between curly braces) ---
def determine_board_eval_group(index: str) -> str:
    boardKit: str = index.split("{")[1][:-1].strip()
    return boardKit if boardKit else "N/A   "
boardKitGrouped = boardCounts.groupby(determine_board_eval_group).sum()
print(boardKitGrouped.sort_values(ascending=False))

Board
Zynq US+      25
Zynq 7000     14
Virtex-7      10
Artix-7        5
Kintex-7       3
Virtex-6       3
Kintex US      2
Alveo          1
Cyclone V      1
Spartan-3A     1
Name: count, dtype: int64

Board
XC7VX690T     10
XC7Z020       10
XCZU3EG        8
XCZU7EV        7
XCZU9EG        5
XC6VLX240T     3
XC7A35T        3
XC7K325T       3
XCZU15EG       3
XCKU040        2
XC7A200T       2
XC7Z045        2
XCZU19EG       1
5CSXC6         1
XCK26          1
XC7Z100        1
U280           1
XC3SD1800A     1
XC7Z035        1
Name: count, dtype: int64

Board
N/A                     16
VC709                    7
ZCU104                   6
UltraZed-EG              4
PYNQ-Z1                  3
Arty-35T                 3
ZCU102                   3
OVC3                     3
Alinx AXU15EG            3
ZC706                    2
Z-turn                   2
AC701                    2
KV260                    2
KCU105                   2
KC705                    2
Ultra96                  1
Qua

### Model data

Model information is already split in 3 Series: `'Model'`, `'Equivalent model'` and `'Backbone'`.
I think grouping by `'Model'`, i.e., how the model is called in the article, makes no sense. However, even if `'Equivalent model'` is a subjective tag decided by myself, this conveys interressant information.
Same for `'Backbone'`.


In [31]:
# ----- Initial dataframe -----
print_pretty_df(datapointsDf[["Model", "Equivalent model", "Backbone"]], max_rows=10)

# ----- Group by "Equivalent model" -----
equivalentModelCounts = datapointsDf["Equivalent model"].value_counts()
print(equivalentModelCounts)
print()

# ----- Group by "Backbone" -----
backbineCounts = (datapointsDf["Backbone"].value_counts().rename(lambda x: "N/A" if x == "" else x))
print(backbineCounts)

+---+-------------------------------+------------------+-------------+
|   |             Model             | Equivalent model |  Backbone   |
+---+-------------------------------+------------------+-------------+
| 0 |           BRAM_DSP            |   Fuzzy ARTMAP   |             |
| 1 |            LUT_MUL            |   Fuzzy ARTMAP   |             |
| 2 | Roller Dung Bettle Clustering |        ML        |             |
| 3 |           CNN2@0.7            |      YOLOv2      | MobileNetv1 |
| 4 |           CNN4@0.7            |      YOLOv2      | MobileNetv2 |
| 5 |           CNN6@1.6            |      YOLOv2      | SqueezeNet  |
| 6 |              MLP              |       MLP        |             |
| 7 |              CNN              |       CNN        |             |
| 8 |             LPDBL             |        ML        |             |
| 9 |           AP2D-Net            |       CNN        |             |
+---+-------------------------------+------------------+-------------+
Equiva

In [32]:
# ----- Find which model is based on MobileNetv2 -----
# If "Model", "Equivalent model" or "Backbone" contains "Mobile"
strLookFor = "tiny"
nb = 0
for index, row in datapointsDf.iterrows():
    if strLookFor in row["Model"] or strLookFor in row["Equivalent model"] or strLookFor in row["Backbone"]:
        print(row["Model"], row["Equivalent model"], row["Backbone"])
        nb += 1
print(f"Number of experiments with {strLookFor} = {nb}")


YOLOv4-tiny 3L YOLOv4 Darknet53-tiny
Improved YOLOv4-tiny YOLOv4-tiny CSPDarknet53
Number of experiments with tiny = 2


### Datasets, RS Applications and ML formulations

In our reporting method, each experiment is performed on a unique Dataset (i.e., we selected the most relevant/common dataset when authors reported results on different one).
Each dataset is used (or even built) for a specific Remote Sensing application which is formulated as a Machine Learning problem or task.

The `Application` tag is no exact science and was kept in order to give some context.


In [33]:
# ----- Print all unique ML problem formulations -----
mlTaskList = datapointsDf["Task"].unique()
print(mlTaskList)

# ----- Group by "Dataset" -----
DatasetCount = datapointsDf["Dataset"].value_counts()
print(DatasetCount)

# ----- Group by "Application" -----
applicationCount = datapointsDf["Application"].value_counts()
print(applicationCount)


['Pixel classification' 'Segmentation' 'Object detection' 'Classification'
 'Regression']
Dataset
University of Pavia {Pixel classification}           6
MSTAR {Classification}                               5
Potsdam {Segmentation}                               4
NWPU-RESISC45 {Classification}                       4
DOTAv1.0 {Object Detection}                          4
SSDD {Object Detection}                              3
AVIRIS-NG {Pixel classification}                     3
UAV RGB (cust.) {Object Detection}                   3
PennSyn2Real {Object Detection}                      3
ALSAT-2A (cust.) {Pixel classification}              2
UAV RGB (cust.) {Pixel classification}               2
RGB (cust.) {Classification}                         2
L8 Biome {Classification}                            2
DIOR {Object Detection}                              2
UAV RGB (cust.) {Classification}                     2
MASATI {Classification}                              2
ALOS-2 (cust.) {Classi

Print unique associations of Task and applications


In [34]:
print_pretty_df(datapointsDf.sort_values(by="Board")[["Board", "Implementation", "Model", "Equivalent model", "Backbone", "Task", "Application"]])

+----+------------------------------------------------+----------------------+-------------------------------+------------------+----------------+----------------------+-----------------------------------------+
|    |                     Board                      |    Implementation    |             Model             | Equivalent model |    Backbone    |         Task         |               Application               |
+----+------------------------------------------------+----------------------+-------------------------------+------------------+----------------+----------------------+-----------------------------------------+
| 48 |                Alveo (U280) {}                 |      HLS (N/A)       |              GNN              |       GNN        |                |    Classification    |     Military targets identification     |
| 32 |           Artix-7 (XC7A200T) {AC701}           |  RTL design (VHDL)   |        Improved YOLOv2        |      YOLOv2      |   Darknet19    |   Obj

In [35]:
# Print unique associations of Task and applications
taskApplicationPairs = datapointsDf[["Task", "Modality", "Application"]].value_counts()
print(taskApplicationPairs.sort_index())

Task                  Modality        Application                            
Classification        1D              Redshift estimation [regression]           1
                      HSI             Air Quality Monitoring                     1
                                      Cloud coverage classification              1
                      RGB             Cloud coverage classification              4
                                      Ship identification                        3
                                      Terrain identification                     6
                      SAR             Military targets identification            5
                                      Ship identification                        2
Object detection      RGB             Aircraft detection                         2
                                      Diverse                                    9
                                      Flying-object detection                    5
         

## Deeper analysis of common dataset and tasks among the dataframe

I try to check how many models (and papers) use the same datasets


In [36]:
# For the first 4 datasets, print each model's name, the corresponding article research key and their metric
for dataset in DatasetCount.index[:4]:
    print(f"\n{b}{dataset}{e} dataset:")
    for index, row in datapointsDf.iterrows():
        if row["Dataset"] == dataset:
            print(
                f"    - {r}{row['Model']}{e} from {b}{row['BBT Citation Key']}{e} with board: {r}{row['Board']}{e} ({b}{row['Implementation']}{e}):"
            )
            print(
                f"        Score: {g}{row['Task score']}{e}, Size: {g}{row['Footprint']}{e}, Latency: {g}{row['Latency']}{e}, Throughput: {g}{row['Throughput']}{e}, Power: {g}{row['Power consumption']}{e}"
            )


University of Pavia {Pixel classification} dataset:
    - LPDBL from shibiOnboardTargetDetection2021a with board: Virtex-7 (XC7VX690T) {} (RTL design (VHDL)):
        Score: , Size: , Latency: , Throughput: , Power: 
    - SAM-GNN from chellaswamyFPGAbasedRemoteTarget2024 with board: Virtex-7 (XC7VX690T) {} (RTL design (VHDL)):
        Score: 95.05% OA, Size: , Latency: , Throughput: , Power: 
    - 2D CNN from heConfigurable2D3D2023a with board: Zynq US+ (XCZU15EG) {Alinx AXU15EG} (RTL design (N/A)):
        Score: 98.24% OA, Size: 1.20 MB, Latency: 0.097 ms, Throughput: 7.07 GOP/s, Power: 8.40 W
    - 3D CNN from heConfigurable2D3D2023a with board: Zynq US+ (XCZU15EG) {Alinx AXU15EG} (RTL design (N/A)):
        Score: 94.09% OA, Size: 0.12 MB, Latency: 1.11 ms, Throughput: 3.81 GOP/s, Power: 8.40 W
    - HybridSN from heConfigurable2D3D2023a with board: Zynq US+ (XCZU15EG) {Alinx AXU15EG} (RTL design (N/A)):
        Score: 100% OA, Size: 20.50 MB, Latency: 7.71 ms, Throughput: 13.18

### Check common tasks


In [37]:
# For each Task, group by dataset and print each model name and backbones
MLTaskCount = datapointsDf["Task"].value_counts()
for task, value in MLTaskCount.items():
    print(f"\n{b}{task}{e} task (total: {value}):")
    nbItems = 0
    for dataset in DatasetCount.index:
        if dataset.split("{")[1][:-1].strip().upper() == task.upper():
            print(f"    {b}{dataset.split('{')[0].strip()}{e}:")
            for index, row in datapointsDf.iterrows():
                if row["Task"] == task and row["Dataset"] == dataset:
                    print(
                        f"        - {r}{row['Model']}{e} ({row['Equivalent model']}) [{y}{row['Backbone']}{e}] from {b}{row['BBT Citation Key']}{e} with board: {r}{row['Board']}{e} ({row['Implementation']}):"
                    )
                    print(
                        f"            Score: {g}{row['Task score']}{e}, Size: {g}{row['Footprint']}{e}, Latency: {g}{row['Latency']}{e}, Throughput: {g}{row['Throughput']}{e}, Power: {g}{row['Power consumption']}{e}"
                    )
                    nbItems += 1

    print(f"    Total: {nbItems}")


Classification task (total: 23):
    MSTAR:
        - LeNet-5 (LeNet-5) [LeNet-5] from chenHardwareImplementationConvolutional2020 with board: Kintex-7 (XC7K325T) {} (N/A):
            Score: 98.18% OA, Size: , Latency: 2.29 ms, Throughput: , Power: 
        - LeNet-5 Q32 (LeNet-5) [LeNet-5] from weiFPGABasedHybridTypeImplementation2019 with board: Kintex-7 (XC7K325T) {KC705} (RTL design (N/A)):
            Score: 98.76% OA, Size: 6.64 MB, Latency: 2.29 ms, Throughput: , Power: 
        - LeNet-5 Q8 (LeNet-5) [LeNet-5] from weiFPGABasedHybridTypeImplementation2019 with board: Kintex-7 (XC7K325T) {KC705} (RTL design (N/A)):
            Score: 97.77% OA, Size: 1.66 MB, Latency: 2.29 ms, Throughput: , Power: 
        - GNN (GNN) [] from zhangAccurateLowlatencyEfficient2022a with board: Zynq US+ (XCZU7EV) {ZCU104} (HLS (N/A)):
            Score: 99.09% OA, Size: 0.96 MB, Latency: 0.105 ms, Throughput: , Power: 6.30 W
        - GNN (GNN) [] from zhangAcceleratingGNNbasedSAR2023 with board:

## Analyze reporting: missing metrics


In [38]:
def is_undefined(item) -> bool:
    if isinstance(item, str):
        return item.startswith("N/A") or item.startswith("???") or item == ""
    elif isinstance(item, list):
        return all(is_undefined(subitem) for subitem in item)
    else:
        raise ValueError(f"Unsupported type: {type(item)}")
    
# ---  Quick check if any of the main information is missing ---
for index, article in datapointsDf.iterrows():
    if is_undefined(article["Model"]):
        print(f"Item N°{b}{index}{e} has no Model")
    if is_undefined(article["Dataset"]):
        print(f"Item N°{b}{index}{e} has no Dataset")
    if is_undefined(article["Board"]):
        print(f"Item N°{b}{index}{e} has no Board")
    if is_undefined(article["Task"]):
        print(f"Item N°{b}{index}{e} has no Task")

### Check per article: Which article miss the most metrics

#### First for the "performance" metrics


In [40]:
performanceMetrics: list[str] = [
    "Latency",
    "Task score",
    "Footprint",
    "Throughput",
    # "Frequency",
    "Complexity",
    "Power consumption",
]
# --- Compute the number of missing (performance) metrics for each model ---
# Add a column to the dataframe with the number of missing metrics
datapointsDf["Missing perf metrics"] = datapointsDf.apply(
    lambda article: sum(
        [
            is_undefined(article[metric])
            for metric in performanceMetrics
        ]
    ),
    axis=1,
)
print(f"{len(datapointsDf[datapointsDf['Missing perf metrics'] >= 1])}/{len(datapointsDf)}")
# Print average number of missing metrics
print(
    f"Average number of missing metrics: {datapointsDf['Missing perf metrics'].mean()}"
)

# --- Compute the number of missing (performance) metrics for each study ---
# Group by the citation key, aggregating the total missing metrics per study
missing_per_study = (
    datapointsDf
    .groupby("BBT Citation Key", as_index=False)["Missing perf metrics"].mean()
)
# Print the fraction of studies with any missing metrics
print(
    f"{len(missing_per_study[missing_per_study['Missing perf metrics'] >= 1])}"
    f"/{len(missing_per_study)} studies have missing metrics"
)
# Print the fraction of studies with any missing metrics
print(
    f"{len(missing_per_study[missing_per_study['Missing perf metrics'] >= 2])}"
    f"/{len(missing_per_study)} studies have more than 1 missing metrics"
)
# Print the fraction of studies with any missing metrics
print(
    f"{len(missing_per_study[missing_per_study['Missing perf metrics'] >= 3])}"
    f"/{len(missing_per_study)} studies have more than 2 missing metrics"
)
# Print the average number of missing metrics per study
print(
    f"Average number of missing metrics (per study): "
    f"{missing_per_study['Missing perf metrics'].mean()}"
)

53/65
Average number of missing metrics: 2.230769230769231
40/45 studies have missing metrics
32/45 studies have more than 1 missing metrics
22/45 studies have more than 2 missing metrics
Average number of missing metrics (per study): 2.588888888888889


#### Then for the "FPGA" metrics


In [40]:
FPGAMetrics: list[str] = [
    "Design",
    "Memory",
    "Precision",
    "Optimizations",
    "FPGA Util",
]
DPUMetrics: list[str] = [
    "Precision",
    "DPU Config",
    "DPU Core",
    "DPU Optimizations",
    "DPU Util",
]

# --- Compute the number of missing (fpga) metrics for each model ---
datapointsDf["Missing fpga metrics"] = datapointsDf.apply(
    lambda article: sum(
        [
            is_undefined(article[metric])
            for metric in FPGAMetrics
        ]
    ),
    axis=1,
)
datapointsDf["Missing dpu metrics"] = datapointsDf.apply(
    lambda article: sum(
        [
            is_undefined(article[metric])
            for metric in DPUMetrics
        ]
    ),
    axis=1,
)
# Print only the 'BBT Citation Key' and the missing metrics, only if there is more than 3 missing metrics
print_pretty_df(
    datapointsDf[
        ["BBT Citation Key", "Missing fpga metrics", "Missing dpu metrics"]
    ]
)

+----+-------------------------------------------------------+----------------------+---------------------+
|    |                   BBT Citation Key                    | Missing fpga metrics | Missing dpu metrics |
+----+-------------------------------------------------------+----------------------+---------------------+
| 0  |          gargAircraftDetectionSatellite2024           |          2           |          4          |
| 1  |       upadhyayDesignImplementationCNNbased2024        |          4           |          3          |
| 2  |       upadhyayDesignImplementationCNNbased2024        |          4           |          3          |
| 3  |       weiFPGABasedHybridTypeImplementation2019        |          0           |          4          |
| 4  |       weiFPGABasedHybridTypeImplementation2019        |          0           |          4          |
| 5  |          torresCombinedWeightlessNeural2020           |          3           |          4          |
| 6  |      chenHardwareImpl

### Check per metric: Which metric are the less reported


In [36]:
allMetrics = performanceMetrics + FPGAMetrics + DPUMetrics
for metric in allMetrics:
    missing_metrics = 0
    for index, article in datapointsDf.iterrows():
        if is_undefined(article[metric]):
            missing_metrics += 1

    print(f"{r}{missing_metrics}{e} models miss the {b}{metric}{e} metric.")

8 models miss the Latency metric.
7 models miss the Task score metric.
34 models miss the Footprint metric.
40 models miss the Throughput metric.
12 models miss the Frequency metric.
34 models miss the Complexity metric.
19 models miss the Power consumption metric.
17 models miss the Design metric.
17 models miss the Memory metric.
2 models miss the Precision metric.
27 models miss the Optimizations metric.
17 models miss the FPGA Util metric.
2 models miss the Precision metric.
54 models miss the DPU Config metric.
61 models miss the DPU Core metric.
59 models miss the DPU Optimizations metric.
60 models miss the DPU Util metric.
